# 컨텐츠 기반 영화 추천 시스템 구축

- 팀 이름: MM-Up
- 팀 구성원: 송현달, 유병욱, 지영석
<br />

- 프로젝트 간략 개요: 사용자가 한글로 영화 제목을 검색하면, 유사도를 기반으로 연관성을 갖는 영화 상위 10개를 추천해주는 시스템을 구축했음. 또한 연도별 정렬, 장르 기반 추천, 키워드 기반 추천 기능 모두 구현했음.

```
프로젝트 프로세스 구성!

1. 데이터 스크래핑 (유병욱) - BS4와 requests모듈을 사용한 영화 키워드, 제목, 장르 크롤링했음.

2. 데이터 전처리 (송현달) - 수집한 데이터를 바탕으로 특수 문자 제거-영화 출시 연도 컬럼 생성-불필요한 문자열 제거 등의 전처리를 진행했음.

3. 모델링 (지영석) - 전처리 완료된 데이터를 바탕으로, 컨텐츠 기반 필터링 알고리즘을 사용해서 추천시스템을 구축했음.

4. 시각화 (송현달) - 영화 장르 키워드 컬럼을 바탕으로, 워드 클라우드 시각화 구축
```

# 1. 크롤링

In [ ]:
# 크롤링 하기위한 import선언
import requests
from bs4 import BeautifulSoup
# 크롤링 후 csv파일로 저장하기 위한 import
import csv
import pandas as pd 

# 나중에 페이지가 늘어날 것을 대비하여 만든 것, 마지막 페이지의 수를 int변수로 저장한다.
page_response = requests.get('https://moviekeyword.com')
page_soup = BeautifulSoup(page_response.text, 'html.parser')
last_page = page_soup.select_one('#main > div > nav > div > a:nth-child(8)')
last = last_page.get_text().replace(',','')
int_last = int(last)

# start_url뒤에 페이지 수를 추가해주면 그 페이지로 이동한다.
start_url = 'https://moviekeyword.com/page/'

def page_jump(n):
    movies_data = []
    for i in range(50*(n) + 1, 50*(n+1)+1): #뒤의 if문과 연관시켜서 값을 넣으면 된다.
        page_num = i
        URL = start_url + str(page_num)
        response = requests.get(URL)

        soup = BeautifulSoup(response.text, 'html.parser')

        # 여기까지 페이지 관련으로 for문을 받아 한바퀴를 돌때마다, 다음 페이지로 이동

        # title 추출 시작
        movies_page_list = soup.select('div[id=page] > div[id=content-area] > div[id=content] > div[id=primary] > main[id=main] > div[id=primary] > main[id=main] > article')

        for movie_page in movies_page_list:
            a_tag = movie_page.select_one('div.post-summary-container.col-md-11 > header > h2 > a')
            movies_title = a_tag.get_text() # title 추출
            movies_link = a_tag['href'] # 각 페이지에 접속하여 content와 keyword를 추출하기 위하여 주소 추출

            # 여기까지 title 추출하기

            # 추출한 링크로 이동하기 위한 response를  mv_response로 설정
            mv_response = requests.get(movies_link)
            new_soup = BeautifulSoup(mv_response.text, 'html.parser')

            # 중간에 content가 5번 위치일 경우 사용할 경로
            movie_content = new_soup.select_one('#right-sidebar > div.right-sidebar-meta-container > ul > li:nth-child(5) > div.meta-value-container > span.meta-value > a')
            # 5번 위치가 아니라 4번에 있는 경우 사용할 경로
            movie_content_reserve = new_soup.select_one('#right-sidebar > div.right-sidebar-meta-container > ul > li:nth-child(4) > div.meta-value-container > span.meta-value > a')
            
            #  위에서 설명한데로 5번이 비어있는경우 4번에서 값을 가지고 온다는 if문
            if movie_content != None:
                content = movie_content.get_text()
            elif movie_content_reserve != None:
                content = movie_content_reserve.get_text()
            else:
                content = '없음'
            
            # 여기까지 장르(content) 추출하기

            movie_keywords = new_soup.select('#right-sidebar > div.right-sidebar-meta-container > ul > li:nth-last-child(1) > div.meta-value-container > span.meta-value')
            
            keywords = []
            # for 문으로 keyword 값들을 각각 가지고 와서 keywords에 append시킨다.
            for movie_keword in movie_keywords:
                keyword_tag = movie_keword.select_one('a')
                # if문은 keyword가 없는 경우 키워드가 없다는 것을 출력시키기 위한 것
                if keyword_tag != None: 
                    keyword = keyword_tag.get_text().strip()
                else:
                    keyword = '없음'
                keywords.append(keyword)

            # 여기까지 키워드 추출하기

            movie_data = {
                'title': movies_title,
                'content': content,
                'keyword': keywords
            }
            movies_data.append(movie_data)
            # print(movie_data) # 영화 제목, 장르, 키워드 출력
            # print(movies_title) #영화 제목 출력
            # print(content) # 영화 장르 출력
            # print(keywords) # 영화 키워드 출력
            print(i, " ", movie_data) 
            # 에러값의 원인을 찾기위해 페이지 값과 영화데이터 값을 각각 출력
            # 일단 출력이 되었으면 거기까지는 문제가 없다는 뜻이다.

        # 위의 첫 for문과 연계 너무 많이하면 오류발생되서 데이터가 하나도 저장이 안됨
        # if문을 사용하여 20페이지 당 파일 이름을 바꾸어서 데이터를 새로 저장
        if i % 50 == 0 or i == int_last:
            dataframe = pd.DataFrame(movies_data)
            dataframe.to_csv(f"./save_data_{n}.csv",encoding='UTF-16',header=False,index=False)

n = 0 # 해당 폴더에서 출력되지 않은 숫자를 입력하면 그것부터 다시 출력시킨다.
       
for a in range(n+1,(int_last//50)+2):
    page_jump(n)
    n += 1

# 2. 전처리

In [ ]:
import pandas as pd
from tabulate import tabulate
# csv파일 편집을 위해 pandas 호출
# 테이블을 호출했을때 가독성을 높이기 위한 tabulate 호출

data = pd.read_csv("C:/Users/admin/AI/miniproject/movie_data_before.csv", encoding = 'utf-8') 
# pandas로 csv파일을 읽어준다.(csv경로 재설정 확인 필수)

print(tabulate(data.head(), headers='keys', tablefmt='psql'))
print(tabulate(data.tail(), headers='keys', tablefmt='psql'))
# 제대로 불러왔는지 확인한다.
print(data.shape)
# 데이터프레임의 차원을 확인한다.
print(data.dtypes)
# 컬럼의 타입을 확인한다.

In [ ]:
year = data['title']
# year에 title컬럼값을 저장한다.
year = year.str.replace(pat=r'(.*?)\,\s', repl=r'', regex=True)
# year에서 ', '를 포함한 이전 문자열 모두 제거한다.
year = year.str.replace(pat=r'(.*?)\(', repl=r'', regex=True)
# year에서 '('를 포함한 이전 문자열 모두 제거한다.
year = year.str.replace(pat=r'[\D]', repl=r'', regex=True)
# year에서 숫자를 제외한 모든 문자열을 제거한다.
year = year.replace(r'', r'2010', regex=True)
# year에서 완전히 공백인 공간은 '2010'으로 대체한다.

print(year)
# 결과를 확인한다.

In [ ]:
data = pd.DataFrame(data,columns=['title', 'year', 'genres', 'keywords'])
# 데이터프레임에 'year'컬럼을 추가해준다.
data['year'] = year
# 'year'컬럼에 year값을 저장한다.
print(tabulate(data.head(), headers='keys', tablefmt='psql'))
print(tabulate(data.tail(), headers='keys', tablefmt='psql'))
# 결과를 확인한다.

In [ ]:
data[data['year'].str.len() != 4]
# 'year'컬럼값이 길이4가 아닌 행을 확인한다.

In [ ]:
for row in range(len(data.year)):
    data['year'] = data['year'].str[-4:]
    #'year'컬럼값을 뒤에서 길이4로 슬라이싱한다.

data = data[data['year'].str.len() == 4]
# 'year'컬럼값이 길이4가 아닌 행을 제거한다.

data[data['year'].str.len() != 4]
# 'year'컬럼값이 길이4가 아닌 행을 확인한다.

In [ ]:
data['title'] = data['title'].str.replace(pat=r'\((.*?)\)', repl=r' ', regex=True)
# 'title'컬럼에서 소괄호와 그 사이의 문자를 모두 공백으로 바꾼다.

print(data['title'])
# 처리된 'title'컬럼을 확인한다.

In [ ]:
data['keywords'] = data['keywords'].str.replace(pat=r'[^\w\s]', repl=r' ', regex=True)
# 'keywords'컬럼에서 공백, 알파벳, 숫자가 아닌 문자를 모두 공백으로 바꾼다.
data = data[data['keywords'] != '  없음  ']
# 'keywords'컬럼이 '  없음  '으로 채워진 행을 제거한다.
# (원래 ['없음']이라고 적혀있는 값에서 특수문자를 모두 공백으로 바꿨으므로
# 양끝에 공백을 2개씩 넣어줘야 정상적으로 인식한다.)

print(data['keywords'])
# 처리된 'keywords'컬럼을 확인한다.

In [ ]:
data = data.reset_index()
# 데이터 프레임의 인덱스를 리셋해주고 중간중간 빈 인덱스를 컬럼으로 빼준다.
print(tabulate(data.head(), headers='keys', tablefmt='psql'))
print(tabulate(data.tail(), headers='keys', tablefmt='psql'))
# 결과를 확인한다.

In [ ]:
data.drop(['index'], axis='columns', inplace=True)
# 'index'컬럼으로 빼낸 값을 제거한다.
print(tabulate(data.head(), headers='keys', tablefmt='psql'))
print(tabulate(data.tail(), headers='keys', tablefmt='psql'))
print(data.shape)
# 결과를 확인한다.

In [ ]:
data['year'] = data['year'].astype(str).astype(int)
# 정렬을 용이하게 하기위해서 'year'컬럼을 정수형으로 변환해준다.
print(data.dtypes)
# 컬럼의 타입을 확인한다.

In [ ]:
data.to_csv('movie_data_after.csv')
# 결과물을 'movie_data_after.csv'파일로 저장한다.

# 3. 모델링

In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def tf_idf_matrics(column):  #tf-idf사용해서 문장-단어 간 유사도 측정 함수
    tfidf_vec = TfidfVectorizer(ngram_range=(1,2))
    tfidf_mat = tfidf_vec.fit_transform(column.values.astype("U"))
    return tfidf_mat

# tf_idf_mat = tf_idf_matrics(df["keywords"])
# print(tf_idf_mat)

def cosine_sim_matrics(tfidf_mat): #코사인 유사도 측정 함수
    cos_sim = cosine_similarity(tfidf_mat, tfidf_mat)
    sim_idx = np.argsort(-cos_sim)
    return sim_idx

# genres_sim, sim_idx = cosine_sim_matrics(tf_idf_mat)
# print(genres_sim, sim_idx)

def recommandation_mvs_top10(inputs_mv, sim_idx, df): #input을 사용자로부터 받고, 유사도가 제일 높은 10개의 영화를 추천해주는 함수
#검색한 영화의 인덱스를 찾아준다.
    mv_idx = df[df.title==inputs_mv.lstrip().rstrip()].index.values
    print(mv_idx)
    
#인덱스 값을 기준으로 유사도가 높은 영화 인덱스를 출력한다.
    sim_mv = sim_idx[mv_idx, :int(10)]
    print(sim_mv)

    sim_mv_idx = sim_mv.reshape(-1) #2차원 벡터 -> 1차원으로 변경
    print(sim_mv_idx)
    #x = df.iloc[sim_mv_idx]
    
    top_10_title = pd.Series(df.iloc[sim_mv_idx, 0])
    top_10_years = pd.Series(df.iloc[sim_mv_idx, 1])
    top_10_genres = pd.Series(df.iloc[sim_mv_idx, 2])
    top_10_keywords = pd.Series(df.iloc[sim_mv_idx, 3])

    recommandation_df = pd.DataFrame({
        "title": top_10_title,
        "year": top_10_years,
        "genres": top_10_genres,
        "keywords": top_10_keywords
    })
    
    return recommandation_df

# inputs_mv = input()
# r = recommandation_mvs_top10(inputs_mv, sim_idx)
# print(r)

# def main1():
#     drive.mount("/content/gdrive")
#     df = pd.read_csv("/content/gdrive/My Drive/광인사 자연어 프로젝트/final_prepro_data2.csv")
#     tf_idf_mat = tf_idf_matrics(df["keywords"])
#     print(f'TF-IDF-VECTOR- \n {tf_idf_mat}')
#     genres_sim, sim_idx = cosine_sim_matrics(tf_idf_mat)
#     print(f"\n COSINE-VECTOR- \n {genres_sim}")
#     print(f"\n SORTED-COSINE-VECTOR- \n {genres_sim}")
#     inputs_mv = input()
#     r = recommandation_mvs_top10(inputs_mv, sim_idx)
#     print(r)

def main2():
    drive.mount("/content/gdrive")
    #data = pd.read_csv("/content/gdrive/My Drive/mvs_final_csvfile.csv")
    data = pd.read_csv("recomm_data.csv")
    df = data.drop("Unnamed: 0", axis=1)
    inputs_mv = input("---찾고자 하는 영화를 입력해주세요!--- \n")
    input_method_of_recommandation = int(input('''
                    검색하시는 영화 이외에, 회원님이 좋아하실만한 영화 10개를 더 추천해드리려고 합니다! \n
                    1. 비슷하거나 같은 장르의 영화를 추천받고 싶으신가요? \n
                    2. 비슷하거나 같은 키워드나 주제를 가진 영화를 추천받고 싶으신가요? \n
                            '''))
    
    if input_method_of_recommandation == 1:
        tf_idf_mat = tf_idf_matrics(df["genres"])
    elif input_method_of_recommandation == 2:
        tf_idf_mat = tf_idf_matrics(df["keywords"])
    else:
        print("다시 선택해주세요!")

    #print(f'TF-IDF-VECTOR- \n {tf_idf_mat}')
    sim_idx = cosine_sim_matrics(tf_idf_mat)
    #print(f"\n COSINE-VECTOR- \n {genres_sim}")
    #print(f"\n SORTED-COSINE-VECTOR- \n {genres_sim}")
    print("---처리 중 입니다!--")
    recomm_df = recommandation_mvs_top10(inputs_mv, sim_idx, df)
    recomm_df = recomm_df.sort_values(by="year", ascending=False)
    recomm_df.to_csv("recommand_top_10_mvs.csv", index=False)

if __name__ == "__main__":
    main2()

# 4. 시각화

In [ ]:
from konlpy.tag import Twitter
from collections import Counter
import pandas as pd
# 한글로 이루어진 키워드를 처리하기 위해서 konlpy의 Twitter를 호출해준다.
# 키워드의 빈도수를 측정하기 위해 collections의 Counter를 호출해준다.
# csv파일을 편집하기위해 pandas를 호출해준다.

file = pd.read_csv("C:/Users/admin/AI/miniproject/movie_data_after.csv", encoding = 'utf-8')
# csv파일을 호출한다.
kw_file = file.keywords
# 'keywords'컬럼만 따로 불러온다.
print(kw_file)
# 결과를 확인한다.

In [ ]:
kw_file = kw_file.str.strip()
# 컬럼값 양쪽의 공백을 제거한다.
kw_file = kw_file.str.replace(pat=r'\s+', repl=r' ', regex=True)
# 컬럼값 가운데 중복된 공백을 한개로 줄인다.
print(kw_file)
# 결과를 확인한다.

In [ ]:
kw_file = kw_file.to_list()
# 시리즈 형식인 kw_file을 리스트화한다.
kw_file = ' '.join(kw_file)
# 리스트가 된 kw_file을 str으로 변환하여 ' '을 기준으로 split한다. 
kw_file = kw_file.split(' ')
# str인 kw_file을 리스트로 변환하여 ' '을 기준으로 split한다.
print(kw_file)
# 결과를 확인한다.

In [ ]:
count = Counter(kw_file)
# kw_file내 단어들의 빈도수를 계산한다.
words = dict(count.most_common())
# 빈도수를 계산한 값에서 가장 빈도수가 많은 순으로 정렬하여 dictionary 형태로 만든다.

In [ ]:
from wordcloud import WordCloud 
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rc('font',family = 'Malgun Gothic')
# 워드 클라우드에 필요한 함수들을 호출한다.
# font는 Malgun Gothic

wordcloud = WordCloud(
    font_path = 'C:/Windows/Fonts/malgun.ttf',
    background_color='white',
    colormap = "Accent_r",
     width=1500,
     height=1000
     ).generate_from_frequencies(words)
# font_path에 한글 폰트가 있는 경로를 적어준다. 
# background_color는 'white'로 설정한다.
# 마지막 괄호에 dictionary 형태로 만들었던 words를 넣어준다. 

plt.imshow(wordcloud)
plt.axis('off')
plt.show()